This notebook contains the discovery for how to calculate the CPT for P(O2 Saturation Fully Functional Alveoli | Healthy O2 Saturation, Airway Resistance)

In [1]:
import numpy as np
import sys

sys.path.append("../milestone_model")
import lung_health_models

In [2]:
# How to use reshape to convert a 3D CPT to a 2D that can be ingested by PGMPY?

CPT_i_j_k = np.array([[[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]], [[0.7, 0.8, 0.9], [0.11, 0.12, 0.13]]])
CPT_i_j_k.shape
# Reshape to 2D
CPT_i_j_k.reshape(2, 6)

array([[0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 ],
       [0.7 , 0.8 , 0.9 , 0.11, 0.12, 0.13]])

In [3]:
healthy_FEV1_prior = {
    "type": "gaussian",
    "mu": 3.5,
    "sigma": 0.4,
}
healthy_o2sat_prior = {
    "type": "gaussian",
    "mu": 0.98,
    "sigma": 0.01,
}
(
    model,
    HFEV1,
    prior_hfev1,
    FEV1,
    HO2Sat,
    prior_ho2sat,
    O2SatFFA,
    AR,
    prior_ar,
) = lung_health_models.build_FEV1_O2_point_in_time_model(
    healthy_FEV1_prior, healthy_o2sat_prior
)

*** Building lung model with HFEV1 and AB ***
Defining gaussian prior with mu = 3.50, sigma = 0.4
Defining gaussian prior with mu = 0.98, sigma = 0.01


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../milestone_model/model_helpers.py:289: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(
